In [67]:
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta
import requests
import pandas as pd
import numpy as np
import os.path
import joblib

from urllib3.exceptions import InsecureRequestWarning
# Disable the warning about unverified HTTPS requests
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Set INITIAL TIME & END TIME
INITIAL_TIME = 0
END_TIME = 0

# Set ELASTICSEARCH_URL, ES_USERNAME, ES_PASSWORD
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"
INDEX_NAME = "siem-*"

# orignal dataframe
DF=pd.DataFrame()





# def reset_time():
#     global INITIAL_TIME
#     global END_TIME
    
#     if(INITIAL_TIME != 0):
#         INITIAL_TIME = END_TIME

#     else:
#         last_timestamp = datetime.utcnow() - timedelta(minutes=2)
#         INITIAL_TIME = last_timestamp.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
#     current_time = datetime.utcnow()
#     END_TIME = current_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

# reset_time()
# print(INITIAL_TIME)
# print(END_TIME)

# def find_hits():
#     global INITIAL_TIME
#     global END_TIME
#     global ELASTICSEARCH_URL
#     global ES_USERNAME
#     global ES_PASSWORD

#     url = f"{ELASTICSEARCH_URL}/siem-*/_search"
#     headers = {
#         "Content-Type": "application/json"
#     }
#     payload = {
#         "query": {
#             "range": {
#                 "@timestamp": {
#                     "gte": INITIAL_TIME,
#                     "lte": END_TIME
#                 }
#             }
#         }
#     }

#     try:
#         response = requests.get(url, auth=(ES_USERNAME, ES_PASSWORD), headers=headers, json=payload, verify=False)
#         if response.status_code == 200:
#             logs = response.json()
#             total_logs = logs["hits"]["total"]["value"] if "hits" in logs else 0
#             # print(f"Total new logs since {INITIAL_TIME}: {total_logs}")
#             return total_logs
#         else:
#             print("Failed to fetch logs.")
#     except Exception as e:
#         print(f"Failed to connect: {e}")

# # print(find_hits())

# def add_data(hits):
#     global INITIAL_TIME
#     global END_TIME
#     global ELASTICSEARCH_URL
#     global ES_USERNAME
#     global ES_PASSWORD
#     global INDEX_NAME

#     url = f"{ELASTICSEARCH_URL}/{INDEX_NAME}/_search"
#     headers = {
#         "Content-Type": "application/json"
#     }
#     payload = {
#         "sort": [
#             {
#                 "@timestamp": {
#                     "order": "desc",
#                     "format": "strict_date_optional_time",
#                     "unmapped_type": "boolean"
#                 }
#             },
#             {
#                 "_doc": {
#                     "order": "desc",
#                     "unmapped_type": "boolean"
#                 }
#             }
#         ],
#         "track_total_hits": False,
#         "fields": [
#             {
#                 "field": "*",
#                 "include_unmapped": "true"
#             },
#             {
#                 "field": "@timestamp",
#                 "format": "strict_date_optional_time"
#             },
#             {
#                 "field": "BeginTime",
#                 "format": "strict_date_optional_time"
#             },
#             {
#                 "field": "EndTime",
#                 "format": "strict_date_optional_time"
#             },
#             {
#                 "field": "Time",
#                 "format": "strict_date_optional_time"
#             }
#         ],
#         "size": hits,
#         "version": True,
#         "script_fields": {},
#         "stored_fields": ["*"],
#         "runtime_mappings": {},
#         "_source": False,
#         "query": {
#             "bool": {
#                 "must": [],
#                 "filter": [
#                     {
#                         "range": {
#                             "@timestamp": {
#                                 "format": "strict_date_optional_time",
#                                 "gte": INITIAL_TIME,
#                                 "lte": END_TIME
#                             }
#                         }
#                     }
#                 ],
#                 "should": [],
#                 "must_not": []
#             }
#         },
#         "highlight": {
#             "pre_tags": ["@kibana-highlighted-field@"],
#             "post_tags": ["@/kibana-highlighted-field@"],
#             "fields": {"*": {}},
#             "fragment_size": 2147483647
#         }
#     }

#     response = requests.get(url, auth=(ES_USERNAME, ES_PASSWORD), headers=headers, json=payload, verify=False)
    
    
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return {"error": "Failed to fetch logs."}

# # Call the function to fetch logs
# logs = add_data(find_hits())


    

# # Check if logs were fetched successfully
# if "error" not in logs:
#     hits = logs.get("hits", {}).get("hits", [])
    
#     # Extracting fields from hits
#     data = [hit.get("fields", {}) for hit in hits]

#     # Create DataFrame
#     df_logs = pd.DataFrame(data)

In [68]:
# df_logs.head()

In [69]:
# import pandas as pd

# def clean_data(df_logs):
#     # check for req
#     required_columns = [
#         "@timestamp", "Action", "Application", "Attack", "BeginTime", "Category", "CloseReason", "Cpu",
#         "Destination-address", "Destination-port", "Destination-vpn-id", "Destination-zone", "DstLocation",
#         "EndTime", "EventNum", "IP-address", "IPVer", "MaxSpeed", "ModuleBrief", "ModuleName", "Os", "Policy",
#         "Policy-name", "Priority", "Profile", "Protocol-Name", "Protocol-Number", "RcvBytes", "RcvPkts",
#         "RecieveInterface", "Role", "SendBytes", "SendPkts", "Severity", "SignId", "SignName", "Source-address",
#         "Source-vpn-id", "Source-zone", "SrcLocation", "SyslogId", "Target", "TotalPackets", "UserName", "VSys", "slot"
#     ]
    
#     missing_columns = [col for col in required_columns if col not in df_logs.columns]

#     if missing_columns:
#         print(f"The following columns are missing: {', '.join(missing_columns)}")
        
#         # Add missing columns with NaN values to the DataFrame
#         for col in missing_columns:
#             df_logs[col] = pd.NA  # or df_logs[col] = None for older Pandas versions
    
#     else:
#         print("All required columns are present in the DataFrame.")

# # Example usage
# # Assuming df_logs is your DataFrame
# # clean_data(df_logs)


In [70]:
def add_new_time_values():

    global DF
    
    # def reset_time():
    #     global INITIAL_TIME
    #     global END_TIME
        
    #     if(INITIAL_TIME != 0):
    #         INITIAL_TIME = END_TIME

    #     else:
    #         last_timestamp = datetime.utcnow() - timedelta(minutes=2)
    #         INITIAL_TIME = last_timestamp.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        
    #     current_time = datetime.utcnow()
    #     END_TIME = current_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    # reset_time()
    # print(INITIAL_TIME)
    # print(END_TIME)

    def reset_time():
        global INITIAL_TIME
        global END_TIME
        log_file = "time_log.txt"  # File to store start and end times

        if os.path.exists(log_file):
            # Read the last recorded end time from the log file
            with open(log_file, 'r') as file:
                lines = file.readlines()
                last_end_time = lines[-1].strip() if lines else None

            if last_end_time:
                start_time = datetime.strptime(last_end_time, "%Y-%m-%dT%H:%M:%S.%fZ")
            else:
                start_time = datetime.utcnow() - timedelta(minutes=2)
        else:
            start_time = datetime.utcnow() - timedelta(minutes=2)

        end_time = start_time + timedelta(minutes=2)

        # Store the end time in the log file for future reference
        with open(log_file, 'a') as file:
            file.write(end_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ") + "\n")

        INITIAL_TIME = start_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        END_TIME = end_time.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    reset_time()    



    def find_hits():
        global INITIAL_TIME
        global END_TIME
        global ELASTICSEARCH_URL
        global ES_USERNAME
        global ES_PASSWORD

        url = f"{ELASTICSEARCH_URL}/siem-*/_search"
        headers = {
            "Content-Type": "application/json"
        }
        payload = {
            "query": {
                "range": {
                    "@timestamp": {
                        "gte": INITIAL_TIME,
                        "lte": END_TIME
                    }
                }
            }
        }

        try:
            response = requests.get(url, auth=(ES_USERNAME, ES_PASSWORD), headers=headers, json=payload, verify=False)
            if response.status_code == 200:
                logs = response.json()
                total_logs = logs["hits"]["total"]["value"] if "hits" in logs else 0
                # print(f"Total new logs since {INITIAL_TIME}: {total_logs}")
                return total_logs
            else:
                print("Failed to fetch logs.")
        except Exception as e:
            print(f"Failed to connect: {e}")

    # print(find_hits())

    def add_data(hits):
        global INITIAL_TIME
        global END_TIME
        global ELASTICSEARCH_URL
        global ES_USERNAME
        global ES_PASSWORD
        global INDEX_NAME

        url = f"{ELASTICSEARCH_URL}/{INDEX_NAME}/_search"
        headers = {
            "Content-Type": "application/json"
        }
        payload = {
            "sort": [
                {
                    "@timestamp": {
                        "order": "desc",
                        "format": "strict_date_optional_time",
                        "unmapped_type": "boolean"
                    }
                },
                {
                    "_doc": {
                        "order": "desc",
                        "unmapped_type": "boolean"
                    }
                }
            ],
            "track_total_hits": False,
            "fields": [
                {
                    "field": "*",
                    "include_unmapped": "true"
                },
                {
                    "field": "@timestamp",
                    "format": "strict_date_optional_time"
                },
                {
                    "field": "BeginTime",
                    "format": "strict_date_optional_time"
                },
                {
                    "field": "EndTime",
                    "format": "strict_date_optional_time"
                },
                {
                    "field": "Time",
                    "format": "strict_date_optional_time"
                }
            ],
            "size": hits,
            "version": True,
            "script_fields": {},
            "stored_fields": ["*"],
            "runtime_mappings": {},
            "_source": False,
            "query": {
                "bool": {
                    "must": [],
                    "filter": [
                        {
                            "range": {
                                "@timestamp": {
                                    "format": "strict_date_optional_time",
                                    "gte": INITIAL_TIME,
                                    "lte": END_TIME
                                }
                            }
                        }
                    ],
                    "should": [],
                    "must_not": []
                }
            },
            "highlight": {
                "pre_tags": ["@kibana-highlighted-field@"],
                "post_tags": ["@/kibana-highlighted-field@"],
                "fields": {"*": {}},
                "fragment_size": 2147483647
            }
        }

        response = requests.get(url, auth=(ES_USERNAME, ES_PASSWORD), headers=headers, json=payload, verify=False)
        
        
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "Failed to fetch logs."}

    # Call the function to fetch logs
    logs = add_data(find_hits())


        

    # Check if logs were fetched successfully
    if "error" not in logs:
        hits = logs.get("hits", {}).get("hits", [])
        
        # Extracting fields from hits
        # data = [hit.get("fields", {}) for hit in hits]
        data = [{"_id": hit.get("_id"), **hit.get("fields", {})} for hit in hits]


        # Create DataFrame
        df_logs = pd.DataFrame(data)
        DF = df_logs.copy()
        return df_logs

In [71]:
def create_db():
    #check if seam_logs.db exist or not
    if os.path.exists('seam_logs.db'):
        print("Database already exists.")
    else:
        print("Creating database...")
        import sqlite3

        # Connect to a database (will create a new one if it doesn't exist)
        conn = sqlite3.connect('seam_logs.db')

        # Create a cursor object to execute SQL commands
        cursor = conn.cursor()

        # SQL command to create a table
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS seam_table (
            Action INTEGER,
            Application INTEGER,
            Attack INTEGER,
            Category INTEGER,
            CloseReason INTEGER,
            Cpu REAL,
            "Destination-address" INTEGER,
            "Destination-port" REAL,
            "Destination-vpn-id" REAL,
            "Destination-zone" INTEGER,
            DstLocation INTEGER,
            EventNum REAL,
            "IP-address" INTEGER,
            IPVer REAL,
            MaxSpeed REAL,
            ModuleBrief INTEGER,
            ModuleName INTEGER,
            Os INTEGER,
            Policy INTEGER,
            "Policy-name" INTEGER,
            Priority TEXT,
            Profile INTEGER,
            "Protocol-Name" INTEGER,
            "Protocol-Number" REAL,
            RcvBytes INTEGER,
            RcvPkts INTEGER,
            RecieveInterface INTEGER,
            Role REAL,
            SendBytes INTEGER,
            SendPkts INTEGER,
            Severity TEXT,
            SignId REAL,
            SignName INTEGER,
            "Source-address" INTEGER,
            "Source-vpn-id" REAL,
            "Source-zone" INTEGER,
            SrcLocation INTEGER,
            SyslogId INTEGER,
            Target INTEGER,
            TotalPackets REAL,
            UserName INTEGER,
            VSys INTEGER,
            slot INTEGER,
            Processed REAL,
            DayOfTheWeek INTEGER,
            DayOrNight INTEGER,
            TimeBins INTEGER
        );
        '''

        # Execute the SQL command to create the table
        cursor.execute(create_table_query)

        # Commit changes and close connection
        conn.commit()
        conn.close()


In [72]:
def filter_required_cols(df_logs):
    # Assuming df_logs is your DataFrame
    required_columns = [
        "@timestamp", "Action", "Application", "Attack", "BeginTime", "Category", "CloseReason", "Cpu",
        "Destination-address", "Destination-port", "Destination-vpn-id", "Destination-zone", "DstLocation",
        "EndTime", "EventNum", "IPVer", "MaxSpeed", "ModuleBrief", "ModuleName", "Os", "Policy",
        "Policy-name", "Priority", "Profile", "Protocol-Name", "Protocol-Number", "RcvBytes", "RcvPkts",
        "RecieveInterface", "Role", "SendBytes", "SendPkts", "Severity", "SignId", "SignName", "Source-address",
        "Source-vpn-id", "Source-zone", "SrcLocation", "SyslogId", "Target", "TotalPackets", "UserName", "VSys", "slot"
    ]

    missing_columns = [col for col in required_columns if col not in df_logs.columns]

    if missing_columns:
        print(f"The following columns are missing: {', '.join(missing_columns)}")

        # Add missing columns with NaN values to the DataFrame
        for col in missing_columns:
            df_logs[col] = np.nan

        df_logs = df_logs[required_columns]
    else:
        print("All required columns are present in the DataFrame.")
        df_logs = df_logs[required_columns]
    
    return df_logs




In [73]:
def clean_value_special_symbols(value):
    return str(value).replace("[", "").replace("]", "").replace("'", "").replace("nan", "-")


In [74]:
# from sklearn.preprocessing import LabelEncoder
# import pickle

# def label_encoding(df_logs):
#     # Map of column-wise label encoders
#     label_encoders = {}
    
#     # Load existing label mappings or create an empty one
#     try:
#         with open('label_mapping.pkl', 'rb') as f:
#             label_mapping = pickle.load(f)
#     except FileNotFoundError:
#         label_mapping = {}

#     cols_to_encode = [
#         'Attack', 'Category', 'DstLocation', 'Os', 'SignName', 'SrcLocation', 'Target',
#         'UserName', 'VSys', 'slot', 'Action', 'Policy', 'Profile', 'Protocol-Name',
#         'Application', 'Source-zone', 'CloseReason', 'Destination-zone', 'ModuleName',
#         'ModuleBrief', 'RecieveInterface', 'Policy-name', 'IP-address', 'Source-address', 'Destination-address'
#     ]

#     # Replace specific values in Source-address and Destination-address
#     df_logs['Source-address'] = df_logs['Source-address'].apply(lambda x: '0' if x.startswith('192.168') else x)
#     df_logs['Destination-address'] = df_logs['Destination-address'].apply(lambda x: '0' if x.startswith('192.168') else x)

#     # Apply LabelEncoder to each column in df_logs
#     for col in cols_to_encode:
#         label_encoder = label_mapping.get(col, LabelEncoder())
#         df_logs[col] = label_encoder.fit_transform(df_logs[col])
#         label_encoders[col] = label_encoder
#         label_mapping[col] = label_encoder  # Update or add to the mapping

#     # Save the label mappings for future use
#     with open('label_mapping.pkl', 'wb') as f:
#         pickle.dump(label_mapping, f)

#     return df_logs


# mapping the values


In [75]:
def transforming_columns(df_logs):
    def read_mapping(column_name):
        mapping_dir = 'mapping'
        mapping_file = os.path.join(mapping_dir, f'{column_name}_mapping.txt')
        if not os.path.exists(mapping_file):
            os.makedirs(mapping_dir, exist_ok=True)
            with open(mapping_file, 'w') as file:
                file.write('{}')
        with open(mapping_file, 'r') as file:
            return eval(file.read())

    def write_mapping(column_name, column_mapping):
        mapping_dir = 'mapping'
        mapping_file = os.path.join(mapping_dir, f'{column_name}_mapping.txt')
        with open(mapping_file, 'w') as file:
            file.write(str(column_mapping))

    def transform_column(column_name):
        column_mapping = read_mapping(column_name)
        modified = False  # Flag to track if mapping has been modified

        def assign_unique_number(value):
            nonlocal column_mapping, modified
            if value not in column_mapping:
                unique_number = max(column_mapping.values()) + 1 if column_mapping else 0
                column_mapping[value] = unique_number
                modified = True  # Set flag when a new value is encountered
            return column_mapping[value]

        df_logs[column_name] = df_logs[column_name].apply(assign_unique_number)

        # Write the mapping only if it has been modified
        if modified:
            write_mapping(column_name, column_mapping)

    # Apply transformation to each column
    columns_to_transform = [
            'Attack', 'Category', 'DstLocation', 'Os', 'SignName', 'SrcLocation', 'Target',
            'UserName', 'VSys', 'slot', 'Action', 'Policy', 'Profile', 'Protocol-Name',
            'Application', 'Source-zone', 'CloseReason', 'Destination-zone', 'ModuleName',
            'ModuleBrief', 'RecieveInterface', 'Policy-name'
        ]
    for column in columns_to_transform:
        transform_column(column)
    
    return df_logs



# issues
- Everytime new label ID assigned to every number, even if the number already had labeled
- due to label one number again and again, will cause mis interpret by AI
- We need to adopt some solution to label these values such a way that only new-upcomming value got new unique number, rest not...
- currently IP's columns are changing alot, we need to make different from label technique
- we need to adopt file handling way to store existing labels

# transforming IP-addresses for Unique No

In [76]:
def transform_ip(ip):
    if ip == '-':
        return -1  # Label '-' as -1
    elif ip.startswith('192.168'):
        return 0  # IP starts with 192.168
    else:
        # Remove dots from IP and divide by a hardcoded value (e.g., 10^12)
        return int(ip.replace('.', '')) / 1e12



In [77]:
def replace_values_and_convert_to_numeric1(df):
    columns = [
    'Cpu',
    'Destination-vpn-id',
    'EventNum',
    'IPVer',
    'MaxSpeed',
    'Role',
    'SignId',
    'Source-vpn-id',
    'Protocol-Number',
    'TotalPackets',
    'Destination-port',
    'SendBytes',
    'SendPkts',
    'RcvBytes',
    'RcvPkts',
    'SyslogId',
    'Severity',
    'Priority'
    ]
    for col in columns:
        df[col] = df[col].astype(str)
        df[col] = df[col].replace('-', '-0.1')
        df[col] = pd.to_numeric(df[col], errors='coerce')



In [78]:
# def replace_values_and_convert_to_numeric2(df):

#     columns = [
#     'SendBytes',
#     'SendPkts',
#     'RcvBytes',
#     'RcvPkts',
#     'SyslogId'
#     ]

#     for col in columns:
#         df[col] = df[col].astype(str)
#         df[col] = df[col].replace('-', '0')
#         df[col] = pd.to_numeric(df[col], errors='coerce')



In [79]:
def find_processed_time(df_logs):
    # Replace '-' with a default date in ISO 8601 Zulu format
    default_date = '1970-01-01T00:00:00.000Z'
    df_logs['BeginTime'] = df_logs['BeginTime'].replace('-', default_date)
    df_logs['EndTime'] = df_logs['EndTime'].replace('-', default_date)

    # Convert 'EndTime' and 'BeginTime' columns to datetime format
    df_logs['EndTime'] = pd.to_datetime(df_logs['EndTime'])
    df_logs['BeginTime'] = pd.to_datetime(df_logs['BeginTime'])

    # Calculate time difference in seconds and store in a new 'Processed' column
    df_logs['Processed'] = (df_logs['EndTime'] - df_logs['BeginTime']).dt.total_seconds()

    # Drop 'BeginTime' and 'EndTime' columns
    df_logs = df_logs.drop(columns=['BeginTime', 'EndTime'])
    return(df_logs)



In [80]:
def dig_timestamps(df_logs):
    import pandas as pd

    # Convert to datetime format (Zulu time format)
    df_logs['@timestamp'] = pd.to_datetime(df_logs['@timestamp'])

    # Extracting day of the week
    df_logs['DayOfTheWeek'] = df_logs['@timestamp'].dt.dayofweek  # Numeric representation of day of the week (0 - Monday, 6 - Sunday)

    # Extracting time of the day
    bins = {
        (0, 6): 0,      # Night
        (6, 12): 1,     # Morning
        (12, 18): 2,    # Afternoon
        (18, 24): 3     # Evening
    }
    df_logs['DayOrNight'] = df_logs['@timestamp'].dt.hour.apply(lambda x: next((v for k, v in bins.items() if k[0] <= x < k[1]), 0))

    # Extracting time bins
    bins_labels = ['Late Night', 'Early Morning', 'Morning', 'Afternoon', 'Evening', 'Night']
    bins_edges = [0, 4, 8, 12, 16, 20, 24]  # Customize as needed
    df_logs['TimeBins'] = pd.cut(df_logs['@timestamp'].dt.hour, bins=bins_edges, labels=False)

    df_logs = df_logs.drop(columns=['@timestamp'])
    return(df_logs)



In [81]:
def append_db(df_logs):
    import pandas as pd
    import sqlite3

    # Assuming df_logs is your DataFrame containing the data you want to append to the SQL database
    # And the database file is 'seam_logs.db'

    # Connect to the SQLite database
    conn = sqlite3.connect('seam_logs.db')

    # Append the data from df_logs to the SQL database table 'seam_table'
    df_logs.to_sql('seam_table', conn, if_exists='append', index=False)

    # Commit changes and close connection
    conn.commit()
    conn.close()


In [82]:
def dbscan(df_logs):
    global DF
    # Check for NaN, null, or inf in the entire DataFrame
    problematic_values = df_logs.isnull().values.any() or df_logs.isna().values.any() or df_logs.isin([np.inf, -np.inf]).values.any()

    if problematic_values:
        print("The DataFrame contains NaN, null, or infinite values.")
    else:
        print("The DataFrame does not contain NaN, null, or infinite values.")
        from sklearn.cluster import DBSCAN

        # Assuming df2 contains your DataFrame with all the columns
        X = df_logs.values  # Using all columns as features

        # Initialize DBSCAN
        dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust parameters as needed

        # Fit DBSCAN to your data
        dbscan.fit(X)

        # Retrieve cluster labels and outliers
        labels = dbscan.labels_
        outliers = df_logs[labels == -1]  # Outliers are labeled as -1
        DF['dbscan'] = dbscan.labels_




# added a dbscan() for clustering new upcomming log's data

also appended in the DB datarame

In [83]:
create_db()#1
df_logs=add_new_time_values().copy()#2
df_logs=filter_required_cols(df_logs).copy()#3
df_logs = df_logs.applymap(clean_value_special_symbols)#4
# df_logs=label_encoding(df_logs).copy()#5
# Applying transformation to the 'Destination-address' column
df_logs['Destination-address'] = df_logs['Destination-address'].apply(transform_ip)
df_logs['Source-address'] = df_logs['Source-address'].apply(transform_ip)
# df_logs['IP-address'] = df_logs['IP-address'].apply(transform_ip)#removing it since it has very different information rather then destination and source address
df_logs = transforming_columns(df_logs).copy()#6

replace_values_and_convert_to_numeric1(df_logs)#6
# replace_values_and_convert_to_numeric2(df_logs)#7
df_logs=find_processed_time(df_logs).copy()#8
df_logs=dig_timestamps(df_logs).copy()#9
# append_db(df_logs)#10

#finding time to process below function
import time
start_time = time.time()
dbscan(df_logs)#11
print("--- %s seconds ---" % (time.time() - start_time))

Database already exists.
The following columns are missing: Category, DstLocation, EventNum, Os, Role, SignId, SignName, SrcLocation, Target
The DataFrame does not contain NaN, null, or infinite values.
--- 0.03399777412414551 seconds ---


# rtrying to add in ES

In [20]:
DF

,_id,Policy-name.keyword,ModuleName,Protocol-Name.keyword,Source-port,Time,Protocol-Number,HostName.keyword,host.ip,host.ip.keyword,...,Attack.keyword,MaxSpeed,TotalPackets,RecieveInterface.keyword,Attack,Cpu.keyword,slot.keyword,application-name,application-name.keyword,dbscan
0,tNX6r4wBX92f7DaOi5Py,[Wifi-1_to_Internet. ],[POLICY],[TCP],[38133],[2023-12-28T05:30:24.000Z],[6],[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,s9X6r4wBX92f7DaOi5Py,[LAN-1_to_VMs. ],[POLICY],[UDP],[52761],[2023-12-28T05:30:24.000Z],[17],[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,LaL6r4wBC4-Y0Pl2ixHw,[LAN-2_to_Internet],[SECLOG],[TCP],[62462],NaN,NaN,[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,R2H6r4wBPF9qFOBui3Ts,[Wifi-1_to_VMs. ],[POLICY],[UDP],[50583],[2023-12-28T05:30:24.000Z],[17],[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,RmH6r4wBPF9qFOBui3Ts,[Wifi-1_to_VMs. ],[POLICY],[UDP],[52383],[2023-12-28T05:30:24.000Z],[17],[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2894,OV74r4wBiXstdpQhvI4R,[LAN-2_to_Internet],[SECLOG],[TCP],[50014],NaN,NaN,[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2895,g0f4r4wBlEEDCPw3u4_w,[Wifi-1_to_Internet. ],[POLICY],[TCP],[52731],[2023-12-28T05:28:25.000Z],[6],[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2896,dGH4r4wBPF9qFOBuu3HI,[Wifi-1_to_Internet],[SECLOG],[TCP],[52661],NaN,NaN,[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2897,C9X4r4wBX92f7DaOu0i-,[default. ],[POLICY],[UDP],[38514],[2023-12-28T05:28:25.000Z],[17],[Lynx-Huawei-USG6500E],[192.168.0.254],[192.168.0.254],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17


In [48]:
DF.columns

Index(['log_id', 'Policy-name.keyword', 'ModuleName', 'Protocol-Name.keyword',
       'Source-port', 'Time', 'Protocol-Number', 'HostName.keyword', 'host.ip',
       'host.ip.keyword', 'Source-address.keyword', 'ModuleBrief', '@version',
       'Destination-port', 'Policy-name', 'Protocol-Name', 'HostName',
       'Source-zone', 'VSys.keyword', 'event.original', 'Priority',
       '@version.keyword', 'Severity', 'Destination-address.keyword',
       'message', 'Destination-zone.keyword', 'Source-address',
       'Source-port.keyword', '@timestamp', 'Destination-zone',
       'Destination-address', 'ModuleName.keyword', 'message.keyword',
       'Destination-port.keyword', 'ModuleBrief.keyword', 'Priority.keyword',
       'Source-zone.keyword', 'VSys', 'BeginTime', 'CloseReason',
       'Source-nat-port', 'Source-vpn-id.keyword', 'ApplicationName.keyword',
       'SendPkts', 'EndTime', 'RcvPkts', 'Source-nat-port.keyword',
       'SendBytes', 'IPVer', 'Source-vpn-id', 'ApplicationName',

In [21]:
pip install elasticsearch_dsl

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 63.1/63.1 kB 562.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# checking the connection is establish or not

In [23]:
from elasticsearch import Elasticsearch

# Your Elasticsearch details
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"

# Create an Elasticsearch connection
es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# Check the connection
if es_client.ping():
    print("Elasticsearch connection successful!")
else:
    print("Unable to establish connection to Elasticsearch.")


Elasticsearch connection successful!


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  if __name__ == "__main__":


In [25]:
# from elasticsearch import Elasticsearch
# from elasticsearch import helpers
# from elasticsearch_dsl import connections

# # es_client = connections.create_connection(hosts=['http://localhost:9200/'])
# ELASTICSEARCH_URL = "https://192.168.7.2:9200"
# ES_USERNAME = "elastic"
# ES_PASSWORD = "ncsael@123"

# # Create an Elasticsearch connection
# es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# # def doc_generator(DF):
# #     df_iter = DF.iterrows()
# #     for index, document in df_iter:
# #         yield {
# #                 "_index": 'dbscan_table',
# #                 "_source": document,
# #             }
# def doc_generator(DF, index_name):
#     df_iter = DF.iterrows()
#     for index, document in df_iter:
#         yield {
#             "_index": index_name,
#             "_source": document.to_dict(),  # Convert the row to a dictionary
#         }

        
# helpers.bulk(es_client, doc_generator(DF, "dbscan_table"))


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  # This is added back by InteractiveShellApp.init_path()


BulkIndexError: 500 document(s) failed to index.

# renaming cols's name to avoid conflict of ES default name

In [85]:
# Assuming your DataFrame is named 'DF'
DF.rename(columns={"_id": "log_id"}, inplace=True)


# for just creating a table

In [44]:
from elasticsearch import Elasticsearch

# Your Elasticsearch details
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"

# Create an Elasticsearch connection
es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# Index name to delete
index_name = "logs_index"

# Delete the existing index, if it exists
es_client.indices.delete(index=index_name, ignore=[400, 404])


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  if __name__ == "__main__":
c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  from ipykernel import kernelapp as app


ObjectApiResponse({'error': {'root_cause': [{'type': 'index_not_found_exception', 'reason': 'no such index [logs_index]', 'index_uuid': '_na_', 'resource.type': 'index_or_alias', 'resource.id': 'logs_index', 'index': 'logs_index'}], 'type': 'index_not_found_exception', 'reason': 'no such index [logs_index]', 'index_uuid': '_na_', 'resource.type': 'index_or_alias', 'resource.id': 'logs_index', 'index': 'logs_index'}, 'status': 404})

In [59]:
DF

,_id,Protocol-Name.keyword,BeginTime,host.ip.keyword,CloseReason,Policy-name,HostName,Source-nat-port,event.original,Source-vpn-id.keyword,...,Attack,Cpu.keyword,slot.keyword,Direction.keyword,FileName,Direction,FileName.keyword,FileType.keyword,FileType,dbscan
0,u4_8r4wBj-9RMafbYDvU,[UDP],[2023-12-28T10:31:49.000Z],[192.168.0.254],[aged-out],[VMs_to_Internet],[Lynx-Huawei-USG6500E],[44400],[<190>2023-12-28 10:32:24 Lynx-Huawei-USG6500E...,[0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1,uo_8r4wBj-9RMafbYDuG,[UDP],NaN,[192.168.0.254],NaN,[LAN-1_to_VMs. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:32:24 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,uY_8r4wBj-9RMafbYDsU,[UDP],[2023-12-28T10:31:48.000Z],[192.168.0.254],[aged-out],[LAN-1_to_VMs],[Lynx-Huawei-USG6500E],NaN,[<190>2023-12-28 10:32:24 Lynx-Huawei-USG6500E...,[0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,8178r4wBiXstdpQhXpDh,[TCP],NaN,[192.168.0.254],NaN,[LAN-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:32:24 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,8l78r4wBiXstdpQhXpBz,[TCP],NaN,[192.168.0.254],NaN,[LAN-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:32:24 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,SGH6r4wBPF9qFOBujHTb,[TCP],NaN,[192.168.0.254],NaN,[Wifi-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:30:25 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1925,uF76r4wBiXstdpQhjI_a,[TCP],[2023-12-28T10:29:14.000Z],[192.168.0.254],[tcp-fin],[Wifi-1_to_Internet],[Lynx-Huawei-USG6500E],[2860],[<190>2023-12-28 10:30:25 Lynx-Huawei-USG6500E...,[0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1926,QaL6r4wBC4-Y0Pl2jxLh,[TCP],NaN,[192.168.0.254],NaN,[LAN-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:30:25 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1927,htX6r4wBX92f7DaOjpSk,[TCP],NaN,[192.168.0.254],NaN,[VMs_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:30:25 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


# add table and add data for first time

In [60]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd

# Your Elasticsearch details
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"

# Create an Elasticsearch connection
es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# Index name to create
index_name = "logs_index5"

# Define the index settings and mappings for log_id, dbscan, and @timestamp columns
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "log_id": {"type": "keyword"},
            "dbscan": {"type": "integer"},
            "@timestamp": {"type": "date"}  # Define the @timestamp column here
            # Add more fields and their types as needed
        }
    }
}

# Create the index with the specified settings and mappings
es_client.indices.create(index=index_name, body=index_settings)

# Assuming your DataFrame is named 'DF' and contains the 'log_id', 'dbscan', and '@timestamp' columns
# Replace 'DF' with your actual DataFrame variable
def doc_generator(df):
    for index, row in df.iterrows():
        yield {
            "_index": index_name,
            "_id": row["log_id"],
            "_source": {
                "log_id": row["log_id"],
                "dbscan": int(row["dbscan"]),
                "@timestamp": row["@timestamp"]  # Include the @timestamp column data here
            }
        }

# Index the DataFrame into Elasticsearch
helpers.bulk(es_client, doc_generator(DF[["log_id", "dbscan", "@timestamp"]]))


C:\Users\ZEESHAN\AppData\Roaming\Python\Python37\site-packages\elasticsearch\_sync\client\__init__.py:401: SecurityWarning: Connecting to 'https://192.168.7.2:9200' using TLS with verify_certs=False is insecure
  **transport_kwargs,
c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  # Remove the CWD from sys.path while we load stuff.


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [logs_index5/k5eGIigFT76rtfQs8GpvRA] already exists')

# append data in DBSCAN

In [63]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd

# Your Elasticsearch details
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"

# Create an Elasticsearch connection
es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# Index name to append data
index_name = "logs_index5"  # Use the existing index name

# Assuming your DataFrame is named 'DF' and contains the 'log_id', 'dbscan', and '@timestamp' columns
# Replace 'DF' with your actual DataFrame variable
def doc_generator(df):
    for index, row in df.iterrows():
        yield {
            "_index": index_name,
            "_id": row["log_id"],
            "_source": {
                "log_id": row["log_id"],
                "dbscan": int(row["dbscan"]),
                "@timestamp": row["@timestamp"]
            }
        }

# Index the DataFrame into the existing Elasticsearch index
helpers.bulk(es_client, doc_generator(DF[["log_id", "dbscan", "@timestamp"]]))


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  # Remove the CWD from sys.path while we load stuff.


(1929, [])

# generic:

- to check table already created, then append
- if not already created table, then create and add data

In [86]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd

# Your Elasticsearch details
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"

# Create an Elasticsearch connection
es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# Index name to append data
index_name = "logs_index5"  # Use the existing index name

# Check if the index exists
if not es_client.indices.exists(index=index_name):
    # Define the index settings and mappings for log_id, dbscan, and @timestamp columns
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "log_id": {"type": "keyword"},
                "dbscan": {"type": "integer"},
                "@timestamp": {"type": "date"}
            }
        }
    }

    # Create the index with the specified settings and mappings
    es_client.indices.create(index=index_name, body=index_settings)

# Assuming your DataFrame is named 'DF' and contains the 'log_id', 'dbscan', and '@timestamp' columns
# Replace 'DF' with your actual DataFrame variable
def doc_generator(df):
    for index, row in df.iterrows():
        yield {
            "_index": index_name,
            "_id": row["log_id"],
            "_source": {
                "log_id": row["log_id"],
                "dbscan": int(row["dbscan"]),
                "@timestamp": row["@timestamp"]
            }
        }

# Index the DataFrame into the existing Elasticsearch index
helpers.bulk(es_client, doc_generator(DF[["log_id", "dbscan", "@timestamp"]]))


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  # Remove the CWD from sys.path while we load stuff.


(1428, [])

In [36]:
from elasticsearch import Elasticsearch

# Your Elasticsearch details
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"

# Create an Elasticsearch connection
es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# Index name to create or update
index_name = "logs_index"

# Check if the index exists before deleting it
if es_client.indices.exists(index=index_name):
    # Delete the existing index
    es_client.indices.delete(index=index_name)

# Define the index settings and mappings for _id and dbscan columns
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "_id": {"type": "keyword"},
            "dbscan": {"type": "integer"}
            # Add more fields and their types as needed
        }
    }
}

# Create the index with the specified settings and mappings
es_client.indices.create(index=index_name, body=index_settings)

# ... (Continue with indexing your DataFrame)


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  if __name__ == "__main__":


BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'Field [_id] is defined more than once')

In [41]:
from elasticsearch import Elasticsearch

# Your Elasticsearch details
ELASTICSEARCH_URL = "https://192.168.7.2:9200"
ES_USERNAME = "elastic"
ES_PASSWORD = "ncsael@123"

# Create an Elasticsearch connection
es_client = Elasticsearch([ELASTICSEARCH_URL], http_auth=(ES_USERNAME, ES_PASSWORD), verify_certs=False)

# Index name to delete data from
index_name = "logs_index"

# Define the query to match all documents in the index
delete_query = {
    "query": {
        "match_all": {}
    }
}

# Delete all documents from the logs_index
es_client.delete_by_query(index=index_name, body=delete_query)


c:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  if __name__ == "__main__":


ObjectApiResponse({'took': 3, 'timed_out': False, 'total': 0, 'deleted': 0, 'batches': 0, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [55]:
# Check for NaN, null, or inf in the entire DataFrame
problematic_values = df_logs.isnull().values.any() or df_logs.isna().values.any() or df_logs.isin([np.inf, -np.inf]).values.any()

if problematic_values:
    print("The DataFrame contains NaN, null, or infinite values.")
else:
    print("The DataFrame does not contain NaN, null, or infinite values.")
    from sklearn.cluster import DBSCAN

    # Assuming df2 contains your DataFrame with all the columns
    X = df_logs.values  # Using all columns as features

    # Initialize DBSCAN
    dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust parameters as needed

    # Fit DBSCAN to your data
    dbscan.fit(X)

    # Retrieve cluster labels and outliers
    labels = dbscan.labels_
    outliers = df_logs[labels == -1]  # Outliers are labeled as -1
    DF['dbscan'] = dbscan.labels_




The DataFrame does not contain NaN, null, or infinite values.


In [59]:
DF.head()

,_id,Protocol-Name.keyword,BeginTime,host.ip.keyword,CloseReason,Policy-name,HostName,Source-nat-port,event.original,Source-vpn-id.keyword,...,RecieveInterface,MaxSpeed.keyword,Attack.keyword,MaxSpeed,TotalPackets,RecieveInterface.keyword,Attack,Cpu.keyword,slot.keyword,dbscan
0,J2Hxr4wBPF9qFOBuaGJ9,[TCP],[2023-12-28T10:20:08.000Z],[192.168.0.254],[tcp-rst],[LAN-1_to_Internet],[Lynx-Huawei-USG6500E],[6911],[<190>2023-12-28 10:20:24 Lynx-Huawei-USG6500E...,[0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1,EV7xr4wBiXstdpQhaIIG,[TCP],NaN,[192.168.0.254],NaN,[LAN-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:20:24 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,JmHxr4wBPF9qFOBuZ2L_,[TCP],NaN,[192.168.0.254],NaN,[LAN-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:20:24 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,zkbxr4wBlEEDCPw3ZzTi,[TCP],NaN,[192.168.0.254],NaN,[LAN-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:20:24 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,zUbxr4wBlEEDCPw3ZzTi,[TCP],NaN,[192.168.0.254],NaN,[LAN-1_to_Internet. ],[Lynx-Huawei-USG6500E],NaN,[<190>Dec 28 2023 10:20:24 Lynx-Huawei-USG6500...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [47]:
DF.columns

Index(['_id', 'Protocol-Name.keyword', 'BeginTime', 'host.ip.keyword',
       'CloseReason', 'Policy-name', 'HostName', 'event.original',
       'Source-vpn-id.keyword', 'Priority', '@version.keyword',
       'ApplicationName.keyword', 'SendPkts', 'Source-address',
       'Source-port.keyword', 'ModuleName.keyword', 'ModuleBrief.keyword',
       'Priority.keyword', 'Source-zone.keyword', 'Policy-name.keyword',
       'EndTime', 'ModuleName', 'Source-port', 'HostName.keyword', 'RcvPkts',
       'SendBytes', 'host.ip', 'Source-address.keyword', 'IPVer',
       'ModuleBrief', '@version', 'Destination-port', 'Source-vpn-id',
       'Protocol-Name', 'Source-zone', 'ApplicationName',
       'CloseReason.keyword', 'IPVer.keyword', 'Destination-vpn-id.keyword',
       'Severity', 'Destination-address.keyword', 'message',
       'Destination-zone.keyword', '@timestamp', 'Destination-zone',
       'Destination-address', 'Destination-port.keyword', 'Destination-vpn-id',
       'RcvBytes', 'Time',

In [15]:
df_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5401 entries, 0 to 5400
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Action               5401 non-null   int64  
 1   Application          5401 non-null   int64  
 2   Attack               5401 non-null   int64  
 3   Category             5401 non-null   int64  
 4   CloseReason          5401 non-null   int64  
 5   Cpu                  5401 non-null   float64
 6   Destination-address  5401 non-null   float64
 7   Destination-port     5401 non-null   float64
 8   Destination-vpn-id   5401 non-null   float64
 9   Destination-zone     5401 non-null   int64  
 10  DstLocation          5401 non-null   int64  
 11  EventNum             5401 non-null   float64
 12  IPVer                5401 non-null   float64
 13  MaxSpeed             5401 non-null   float64
 14  ModuleBrief          5401 non-null   int64  
 15  ModuleName           5401 non-null   i

In [25]:
# import pandas as pd
# import os  # Import os module to check file existence

# # Sample DataFrame with 'Attack' column


# # Check if the file exists or it's the first run
# if not os.path.exists('attack_mapping.txt'):
#     with open('attack_mapping.txt', 'w') as file:
#         file.write('{}')  # Initialize an empty dictionary in the file

# # Load or initialize dictionary from a file
# with open('attack_mapping.txt', 'r') as file:
#     attack_mapping = eval(file.read())  # Read the stored dictionary

# def assign_unique_number(value):
#     global attack_mapping
#     if value not in attack_mapping:
#         # Assign a new unique number for the new value
#         unique_number = max(attack_mapping.values()) + 1 if attack_mapping else 0
#         attack_mapping[value] = unique_number
#         # Update the file with the new mapping
#         with open('attack_mapping.txt', 'w') as file:
#             file.write(str(attack_mapping))  # Store the updated dictionary in the file

#     return attack_mapping[value]

In [26]:


# # Applying transformation to the 'Attack' column
# df_logs['Attack-encoded'] = df_logs['Attack'].apply(assign_unique_number)

# # print(df_logs)
# # Saving transformed DataFrame to a CSV file
# df_logs.to_csv('transformed_output.csv', index=False)


In [56]:
# saving to csv
df_logs.to_csv('transformed_output.csv', index=False)

In [35]:
df_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3264 entries, 0 to 3263
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   @timestamp           3264 non-null   object 
 1   Action               3264 non-null   int64  
 2   Application          3264 non-null   int64  
 3   Attack               3264 non-null   int64  
 4   BeginTime            3264 non-null   object 
 5   Category             3264 non-null   int64  
 6   CloseReason          3264 non-null   int64  
 7   Cpu                  3264 non-null   object 
 8   Destination-address  3264 non-null   float64
 9   Destination-port     3264 non-null   object 
 10  Destination-vpn-id   3264 non-null   object 
 11  Destination-zone     3264 non-null   int64  
 12  DstLocation          3264 non-null   int64  
 13  EndTime              3264 non-null   object 
 14  EventNum             3264 non-null   object 
 15  IP-address           3264 non-null   o